## _크롤링_

- resquests : 브라우저가 아닌 코드에서 서버로 리퀘스트를 보내는 것. 
    - 1.API 이용 : request를 던져서 JSON이라는 데이터 포맷(문자열)으로 데이터를 받는다. 문자열을 JSON object로 바꾸어주고 위해 파싱(parsing)을 한다. 그리고 JSON에서 필요한 데이터를 모으면 된다.
    - 2.HTML 코드 이용 : request를 던져서 HTML 코드를 문자열로 받는다. beautifulsoup4(bs4)를 이용해서 파싱(parsing)한다. 그러면 css-selector로 데이터를 모으면 된다. / 정적 웹페이지를 크롤링할 때 사용한다 / 
    - 3.Selenium 이용 : 프론트엔드 페이지 테스팅 도구. 직접 브라우저를 띄우고 css-selector를 이용해서 특정 데이터를 가져온다. / 동적 웹페이지를 크롤링할 때 사용한다
    - 4.scrapy 이용 : 웹 크롤링을 위해 만들어진 파이썬 패키지. request/response 방식. 크롤링 정책을 설정할 수 있다. 동적 페이지를 가져오기 위해서는 Selenium을 사용해야 한다. 



- 우선순위 : 1 > 2 > 3
    - JSON : 필요한 데이터만 집약되어 있다 
    - HTML : JSON에 비해 상대적으로 불필요한 데이터가 있다. request/response로 데이터만 가져온다.
    - Selenium : 브라우저를 띄우고 데이터를 가져오는데 자원이 소모된다.



- 기타 : 화면이 없는 서버에서 오류가 나지 않도록 하기 위해서
    - 크롬드라이버 헤드리스 : 화면을 띄우지 않은 상태에서 크롤링하는 것. Selenium이 이제 더는 phantom.JS를 지원하지 않음.
    - XVfb : 메모리 상 가상화면을 만들어서, 로컬의 화면을 그대로 가져와서 사용할 수 있다.

In [1]:
import requests
from bs4 import BeautifulSoup
import forecastio
import pandas as pd

### *실습*

- api json : 네이버 주식 데이터 크롤링
- api josn / forecastio : dark sky api로 날씨 데이터를 수집
- bs4 : 네이버 실시간 키워드, 다음 실시간 키워드 크롤링
- web file url로 파일을 다운로드 받는 방법

In [ ]:
from pandas.io.json import json_normalize

##### 네이버 주식데이터 가져오기

- 모바일 웹이 크롤링하기 더욱 용이하다

In [ ]:
def make_url(pageSize=10, page=1):
    return "http://m.stock.naver.com/api/json/sise/siseListJson.nhn?\
    menu=market_sum&sosok=0&pageSize=" + str(pageSize) + "&page=" + str(page)
make_url(1458, 1)

In [ ]:
# 데이터를 가져와서 판다스의 데이터프레임으로 만드는 함수
# 1.텍스트 그대로 가져오기
# 2.제이슨 오브젝트로 파싱해서 가져오기

def get_data(url):
    response = requests.get(url)
    json_info = response.json()
    companys = json_info["result"]["itemList"]
    df = pd.DataFrame(columns=["종목", "시세", "전일비", "등락율", "시가총액", "거래량"])
    return json.normalize(companys)
    
#     for company in companys:
#         df.loc[len(df)] = {
#             "종목":company["nm"],
#             "시세":company["nv"],
#             "전일비":company["cv"],
#             "등락율":company["cr"],
#             "시가총액":company["mks"],
#             "거래량":company["aq"],
#         }  
        
#     return df
#     print(response.text) # text 형태로 가져온다
#     print(json_info) # dict 형태로 가져온다
#     print(type(companys)) # List 형태로 가져온다

In [ ]:
url = make_url(1500, 1)
df = get_data(url)
df.tail()

In [ ]:
# 뉴스 크롤링

In [2]:
def make_url(count=10, start=1):
    return "https://m.naver.com/api_airs_news?st=rec&display="\
+ str(count) + "&start=" + str(start)
make_url(100, 1)

'https://m.naver.com/api_airs_news?st=rec&display=100&start=1'

In [3]:
def get_news(url):
    response = requests.get(url)
    json_info = response.json()
    news = json_info["result"]["recommend"]
    df2 = pd.DataFrame(columns=["제목", "언론사", "시간"])
    
    for new in news:
        df2.loc[len(df2)] = {
            "제목":new["title"],
            "언론사":new["office_name"],
            "시간":new["dt_svc"],
        }  
    return df2
news_site = make_url()

get_news(news_site)

TypeError: get_news() missing 1 required positional argument: 'url'

In [ ]:
url = make_url()
df2 = get_data(url)
df2.tail()

### _dark sky api_

- dark sky api : 위도와 경도를 입력하면 날씨 정보를 알려주는 api

In [ ]:
FORECAST_TOKEN = "30dc208a4ebd307081cbb81003d5e455"

In [ ]:
def forecast(lat, lng):
    url = "https://api.darksky.net/forecast/{}/{},{}"\
    .format(FORECAST_TOKEN, lat, lng)
    response = requests.get(url)
    json_info = response.json()
    return json_info["timezone"], json_info["hourly"]["summary"]

In [ ]:
forecast(37.53, 127.04)

### _forecastio_

- 복잡한 api인 경우에는 패키지를 사용하는 것이 좋을 수 있다

In [ ]:
def forecast2(lat, lng):
    forecast = forecastio.load_forecast(FORECAST_TOKEN, lat, lng)
    byHourly = forecast.hourly()
    return byHourly.summary
forecast2(37.53, 127.04)

### _bs4_

- 네이버 키워드 랭킹 데이터 크롤링
- 다음 키워드 랭킹 데이터 크롤링
- dom : HTML 모델
- dom.select : 여러 개의 html 엘리먼트를 셀렉팅할 때 사용. 결과로 리스트 데이터를 리턴한다.
- dom.select_one : 하나의 html 엘리먼트를 셀렉팅할 때 사용. 결과로 문자열 데이터를 리턴한다.

In [ ]:
def naver():
    df = pd.DataFrame(columns=["rank", "keyword"])
    
    response = requests.get("https://www.naver.com/") # requests 가져오기
    dom = BeautifulSoup(response.content, "html.parser") # (markup, feature)
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item") # css-selecting
    
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ah_r").text,
            "keyword": keyword.select_one(".ah_k").text,
        }
    
    return df
naver_df = naver()
naver_df["keyword"]

In [ ]:
import time
ls = []
for idx in range(3):
    ls.append(naver())
    time.sleep(5*1*1)

In [ ]:
ls[0]

### *실습*
    1. daum 실시간 이슈 검색어 10개 수집
    2. 네이버 20개 데이터와 다음 10개 데이터에서 중복되는 데이터 출력

In [ ]:
# 나

def daum():
    df = pd.DataFrame(columns=["rank", "keyword"])
    
    response = requests.get("https://www.daum.net/")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".list_hotissue > roll_txt")

    return len(keywords)
daum()

In [ ]:
# 강사님

def daum():
    df = pd.DataFrame()
    rank = []
    keyword = []
    response = requests.get("https://www.daum.net")
    dom = BeautifulSoup(response.content, 'html.parser')
    keywords = dom.select(".realtime_part .list_hotissue.issue_row > li")

    rank = [ keyword.select_one(".ir_wa").text for keyword in keywords]
    keyword = [ keyword.select_one(".link_issue").text for keyword in keywords]
    
    df["rank"] = rank
    df["keyword"] = keyword
    
    return df

daum_df = daum()
daum_df["keyword"]

In [ ]:
daum_df["keyword"], naver_df["keyword"]

In [ ]:
# 중복값 확인하기

result = [
    keyword
    for keyword in naver_df["keyword"]
    if daum_df["keyword"].str.contains(keyword).any()
]
result

In [ ]:
# set(집합)의 교집합 확인하기

In [ ]:
def naver():
    df = pd.DataFrame(columns=["rank", "keyword"])
    
    response = requests.get("https://www.naver.com/") # requests 가져오기
    dom = BeautifulSoup(response.content, "html.parser") # (markup, feature)
    keywords = dom.select(".ah_roll_area > .ah_l > .ah_item") # css-selecting
    
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank": keyword.select_one(".ah_r").text,
            "keyword": keyword.select_one(".ah_k").text,
        }
    
    return df
naver()

### _file down load : requests_

- file의 url을 받아서 해당 url의 파일을 다운로드하는 방법
- 청크
- 특정 키워드 영상의 url을 가지고 반복적으로 지속가능하게 다운로드할 수 있다


In [ ]:
url = "https://bit.ly/2tbDF7n"
title = "video.mp4"

In [ ]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)
    file_size = 0
    with open(title, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): # chunk의 단위는 byte
            if chunk:
                file_size += 1024
                f.write(chunk)
    return file_size/1024/1024

In [ ]:
download_link = "https://bit.ly/2tbDF7n"
title = "video.mp4"
download(title, download_link)